<a href="https://colab.research.google.com/github/jabanitaha/data-mining-2-/blob/main/M05L01_Lab_Custom_Training_Loops_in_Keras_v1_Answered_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Lab: Custom Training Loops in Keras**


Estimated time needed: **30** minutes


In this lab, you will learn to implement a basic custom training loop in Keras.


## Objectives

By the end of this lab, you will:

- Set up the environment

- Define the neural network model

- Define the Loss Function and Optimizer

- Implement the custom training loop

- Enhance the custom training loop by adding an accuracy metric to monitor model performance

- Implement a custom callback to log additional metrics and information during training


----


## Step-by-Step Instructions:


### Exercise 1: Basic custom training loop:

#### 1. Set Up the Environment:

- Import necessary libraries.

- Load and preprocess the MNIST dataset.


In [9]:
!pip install tensorflow numpy

In [28]:
!pip install keras-tuner --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.1 MB/s eta 0:00:00


In [1]:
import os
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


#### 2. Define the model:

Create a simple neural network model with a Flatten layer followed by two Dense layers.


In [2]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])


#### 3. Define Loss Function and Optimizer:

- Use Sparse Categorical Crossentropy for the loss function.
- Use the Adam optimizer.


In [3]:
# Step 3: Define Loss Function and Optimizer

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


#### 4. Implement the Custom Training Loop:

- Iterate over the dataset for a specified number of epochs.
- Compute the loss and apply gradients to update the model's weights.


In [4]:
# Step 4: Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss

        # Compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Logging the loss every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()}')


Start of epoch 1
Epoch 1 Step 0: Loss = 2.364748954772949
Epoch 1 Step 200: Loss = 0.4014464020729065
Epoch 1 Step 400: Loss = 0.21100589632987976
Epoch 1 Step 600: Loss = 0.18851196765899658
Epoch 1 Step 800: Loss = 0.18301716446876526
Epoch 1 Step 1000: Loss = 0.39724624156951904
Epoch 1 Step 1200: Loss = 0.1480167955160141
Epoch 1 Step 1400: Loss = 0.22252771258354187
Epoch 1 Step 1600: Loss = 0.18365123867988586
Epoch 1 Step 1800: Loss = 0.15444821119308472
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07014244794845581
Epoch 2 Step 200: Loss = 0.15619811415672302
Epoch 2 Step 400: Loss = 0.1362435668706894
Epoch 2 Step 600: Loss = 0.054333437234163284
Epoch 2 Step 800: Loss = 0.09621519595384598
Epoch 2 Step 1000: Loss = 0.2287851721048355
Epoch 2 Step 1200: Loss = 0.11376159638166428
Epoch 2 Step 1400: Loss = 0.13790659606456757
Epoch 2 Step 1600: Loss = 0.12621289491653442
Epoch 2 Step 1800: Loss = 0.06724829971790314


### Exercise 2: Adding Accuracy Metric:

Enhance the custom training loop by adding an accuracy metric to monitor model performance.

#### 1. Set Up the Environment:

Follow the setup from Exercise 1.


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model:
Use the same model as in Exercise 1.


In [6]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define the loss function, optimizer, and metric:

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer.

- Add Sparse Categorical Accuracy as a metric.


In [7]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with accuracy:

Track the accuracy during training and print it at regular intervals.


In [8]:
# Step 4: Implement the Custom Training Loop with Accuracy

epochs = 5  # Number of epochs for training

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)

        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.323561191558838 Accuracy = 0.125
Epoch 1 Step 200: Loss = 0.41141724586486816 Accuracy = 0.8392412662506104
Epoch 1 Step 400: Loss = 0.19501927495002747 Accuracy = 0.8700124621391296
Epoch 1 Step 600: Loss = 0.17804428935050964 Accuracy = 0.8847233653068542
Epoch 1 Step 800: Loss = 0.18697679042816162 Accuracy = 0.8974329233169556
Epoch 1 Step 1000: Loss = 0.38758060336112976 Accuracy = 0.9044080972671509
Epoch 1 Step 1200: Loss = 0.13785453140735626 Accuracy = 0.9110376834869385
Epoch 1 Step 1400: Loss = 0.22846075892448425 Accuracy = 0.9157744646072388
Epoch 1 Step 1600: Loss = 0.21667954325675964 Accuracy = 0.9190545082092285
Epoch 1 Step 1800: Loss = 0.13433201611042023 Accuracy = 0.9229767918586731
Start of epoch 2
Epoch 2 Step 0: Loss = 0.07596714049577713 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.14202824234962463 Accuracy = 0.9625310897827148
Epoch 2 Step 400: Loss = 0.13603036105632782 Accuracy = 0.9590866565704346
Epoch 2 Step 600: Lo

### Exercise 3: Custom Callback for Advanced Logging:

Implement a custom callback to log additional metrics and information during training.

#### 1. Set Up the Environment:

Follow the setup from Exercise 1.


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a batched dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


#### 2. Define the Model:

Use the same model as in Exercise 1.


In [11]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input to a 1D vector
    Dense(128, activation='relu'),  # First hidden layer with 128 neurons and ReLU activation
    Dense(10)  # Output layer with 10 neurons for the 10 classes (digits 0-9)
])


#### 3. Define Loss Function, Optimizer, and Metric:

- Use Sparse Categorical Crossentropy for the loss function and Adam optimizer.

- Add Sparse Categorical Accuracy as a metric.


In [12]:
# Step 3: Define Loss Function, Optimizer, and Metric

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function for multi-class classification
optimizer = tf.keras.optimizers.Adam()  # Adam optimizer for efficient training
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


#### 4. Implement the custom training loop with custom callback:

Create a custom callback to log additional metrics at the end of each epoch.


In [13]:
from tensorflow.keras.callbacks import Callback

# Step 4: Implement the Custom Callback
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')


In [14]:
# Step 5: Implement the Custom Training Loop with Custom Callback

epochs = 2
custom_callback = CustomCallback()  # Initialize the custom callback

for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: Compute predictions
            logits = model(x_batch_train, training=True)
            # Compute loss
            loss_value = loss_fn(y_batch_train, logits)

        # Compute gradients
        grads = tape.gradient(loss_value, model.trainable_weights)
        # Apply gradients to update model weights
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    # Call the custom callback at the end of each epoch
    custom_callback.on_epoch_end(epoch, logs={'loss': loss_value.numpy(), 'accuracy': accuracy_metric.result().numpy()})

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()  # Use reset_state() instead of reset_states()


Start of epoch 1
Epoch 1 Step 0: Loss = 2.3051934242248535 Accuracy = 0.125
Epoch 1 Step 200: Loss = 0.37029320001602173 Accuracy = 0.8347325921058655
Epoch 1 Step 400: Loss = 0.18809539079666138 Accuracy = 0.8673628568649292
Epoch 1 Step 600: Loss = 0.22863906621932983 Accuracy = 0.8826954960823059
Epoch 1 Step 800: Loss = 0.1341085433959961 Accuracy = 0.8955602645874023
Epoch 1 Step 1000: Loss = 0.4830704927444458 Accuracy = 0.9030969142913818
Epoch 1 Step 1200: Loss = 0.16699445247650146 Accuracy = 0.9096326231956482
Epoch 1 Step 1400: Loss = 0.25519728660583496 Accuracy = 0.9147260785102844
Epoch 1 Step 1600: Loss = 0.2104095071554184 Accuracy = 0.9177662134170532
Epoch 1 Step 1800: Loss = 0.14216503500938416 Accuracy = 0.921762228012085
End of epoch 1, loss: 0.056169286370277405, accuracy: 0.92371666431427
Start of epoch 2
Epoch 2 Step 0: Loss = 0.08957361429929733 Accuracy = 0.96875
Epoch 2 Step 200: Loss = 0.1708766371011734 Accuracy = 0.9620646834373474
Epoch 2 Step 400: Loss =

### Exercise 4: Add Hidden Layers

Next, you will add a couple of hidden layers to your model. Hidden layers help the model learn complex patterns in the data.


In [15]:
from tensorflow.keras.layers import Input, Dense

# Define the input layer
input_layer = Input(shape=(28, 28))  # Input layer with shape (28, 28)

# Define hidden layers
hidden_layer1 = Dense(64, activation='relu')(input_layer)  # First hidden layer with 64 neurons and ReLU activation
hidden_layer2 = Dense(64, activation='relu')(hidden_layer1)  # Second hidden layer with 64 neurons and ReLU activation


In the above code:

`Dense(64, activation='relu')` creates a dense (fully connected) layer with 64 units and ReLU activation function.

Each hidden layer takes the output of the previous layer as its input.


### Exercise 5: Define the output layer

Finally, you will define the output layer. Suppose you are working on a binary classification problem, so the output layer will have one unit with a sigmoid activation function.


In [16]:
output_layer = Dense(1, activation='sigmoid')(hidden_layer2)

In the above code:

`Dense(1, activation='sigmoid')` creates a dense layer with 1 unit and a sigmoid activation function, suitable for binary classification.


### Exercise 6: Create the Model

Now, you will create the model by specifying the input and output layers.


In [17]:
model = Model(inputs=input_layer, outputs=output_layer)

In the above code:

`Model(inputs=input_layer, outputs=output_layer)` creates a Keras model that connects the input layer to the output layer through the hidden layers.


### Exercise 7: Compile the Model

Before training the model, you need to compile it. You will specify the loss function, optimizer, and evaluation metrics.


In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In the above code:

`optimizer='adam'` specifies the Adam optimizer, a popular choice for training neural networks.

`loss='binary_crossentropy'` specifies the loss function for binary classification problems.

`metrics=['accuracy']` tells Keras to evaluate the model using accuracy during training.


### Exercise 8: Train the Model

You can now train the model on some training data. For this example, let's assume `X_train` is our training input data and `y_train` is the corresponding labels.


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

# Step 1: Redefine the Model for 20 features
model = Sequential([
    Input(shape=(20,)),  # Adjust input shape to (20,)
    Dense(128, activation='relu'),  # Hidden layer with 128 neurons and ReLU activation
    Dense(1, activation='sigmoid')  # Output layer for binary classification with sigmoid activation
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 2: Generate Example Data
X_train = np.random.rand(1000, 20)  # 1000 samples, 20 features each
y_train = np.random.randint(2, size=(1000, 1))  # 1000 binary labels (0 or 1)

# Step 3: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5109 - loss: 0.6958
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5506 - loss: 0.6878
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5251 - loss: 0.6917
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5392 - loss: 0.6910
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5602 - loss: 0.6869
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5668 - loss: 0.6847
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5587 - loss: 0.6805
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5615 - loss: 0.6825
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5587 - loss: 0.6815
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5571 - loss: 0.6791


In the above code:

`X_train` and `y_train` are placeholders for your actual training data.

`model.fit` trains the model for a specified number of epochs and batch size.


### Exercise 9: Evaluate the Model

After training, you can evaluate the model on test data to see how well it performs.


In [20]:
# Example test data (in practice, use real dataset)
X_test = np.random.rand(200, 20)  # 200 samples, 20 features each
y_test = np.random.randint(2, size=(200, 1))  # 200 binary labels (0 or 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Print test loss and accuracy
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5401 - loss: 0.7020  
Test loss: 0.7010909914970398
Test accuracy: 0.5199999809265137


In the above code:

`model.evaluate` computes the loss and accuracy of the model on test data.

`X_test` and `y_test` are placeholders for your actual test data.


## Practice Exercises

### Exercise 1: Basic Custom Training Loop

#### Objective: Implement a basic custom training loop to train a simple neural network on the MNIST dataset.

#### Instructions:

- Set up the environment and load the dataset.

- Define the model with a Flatten layer and two Dense layers.

- Define the loss function and optimizer.

- Implement a custom training loop to iterate over the dataset, compute the loss, and update the model's weights.


In [21]:
# Write your code here
# Exercise 1: Basic Custom Training Loop

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load and preprocess data
(x_train, y_train), _ = mnist.load_data()
x_train = x_train.astype("float32") / 255.0

# Build model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Training loop
epochs = 5
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if step % 100 == 0:
            print(f"Step {step}: loss = {loss.numpy():.4f}")



Epoch 1/5
Step 0: loss = 2.4150
Step 100: loss = 0.3902
Step 200: loss = 0.3003
Step 300: loss = 0.2694
Step 400: loss = 0.3211
Step 500: loss = 0.1514
Step 600: loss = 0.2177
Step 700: loss = 0.1547
Step 800: loss = 0.1468
Step 900: loss = 0.1159

Epoch 2/5
Step 0: loss = 0.1034
Step 100: loss = 0.1203
Step 200: loss = 0.2074
Step 300: loss = 0.1462
Step 400: loss = 0.0442
Step 500: loss = 0.1879
Step 600: loss = 0.2908
Step 700: loss = 0.3372
Step 800: loss = 0.0346
Step 900: loss = 0.0950

Epoch 3/5
Step 0: loss = 0.0698
Step 100: loss = 0.0680
Step 200: loss = 0.1029
Step 300: loss = 0.0872
Step 400: loss = 0.0957
Step 500: loss = 0.1263
Step 600: loss = 0.0823
Step 700: loss = 0.0418
Step 800: loss = 0.0852
Step 900: loss = 0.0230

Epoch 4/5
Step 0: loss = 0.1762
Step 100: loss = 0.1236
Step 200: loss = 0.0535
Step 300: loss = 0.0396
Step 400: loss = 0.0838
Step 500: loss = 0.1164
Step 600: loss = 0.0838
Step 700: loss = 0.0597
Step 800: loss = 0.0796
Step 900: loss = 0.0240

Epo

<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Step 2: Define the Model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

# Step 3: Define Loss Function and Optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# Step 4: Implement the Custom Training Loop
for epoch in range(5):
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()}')


### Exercise 2: Adding Accuracy Metric

#### Objective: Enhance the custom training loop by adding an accuracy metric to monitor model performance.

#### Instructions:

1. Set up the environment and define the model, loss function, and optimizer.

2. Add Sparse Categorical Accuracy as a metric.

3. Implement the custom training loop with accuracy tracking.


In [22]:
# Write your code here
# Exercise 2: Adding Accuracy Metric

accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    accuracy_metric.reset_state()
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        accuracy_metric.update_state(y_batch, logits)
        if step % 100 == 0:
            print(f"Step {step}: loss = {loss.numpy():.4f}, acc = {accuracy_metric.result().numpy():.4f}")

    print(f"Epoch {epoch + 1} Accuracy: {accuracy_metric.result().numpy():.4f}")



Epoch 1/5
Step 0: loss = 0.0201, acc = 1.0000
Step 100: loss = 0.0455, acc = 0.9878
Step 200: loss = 0.0596, acc = 0.9878
Step 300: loss = 0.0087, acc = 0.9874
Step 400: loss = 0.0505, acc = 0.9875
Step 500: loss = 0.0220, acc = 0.9876
Step 600: loss = 0.0233, acc = 0.9877
Step 700: loss = 0.0121, acc = 0.9876
Step 800: loss = 0.0661, acc = 0.9873
Step 900: loss = 0.0457, acc = 0.9871
Epoch 1 Accuracy: 0.9871

Epoch 2/5
Step 0: loss = 0.0231, acc = 0.9844
Step 100: loss = 0.0081, acc = 0.9895
Step 200: loss = 0.0296, acc = 0.9897
Step 300: loss = 0.0074, acc = 0.9906
Step 400: loss = 0.0299, acc = 0.9900
Step 500: loss = 0.0271, acc = 0.9898
Step 600: loss = 0.0215, acc = 0.9897
Step 700: loss = 0.0240, acc = 0.9895
Step 800: loss = 0.0424, acc = 0.9896
Step 900: loss = 0.0374, acc = 0.9896
Epoch 2 Accuracy: 0.9896

Epoch 3/5
Step 0: loss = 0.0577, acc = 0.9688
Step 100: loss = 0.0611, acc = 0.9899
Step 200: loss = 0.0372, acc = 0.9913
Step 300: loss = 0.0182, acc = 0.9910
Step 400: l

<details>
<summary>Click here for solution</summary><br>

```python
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Step 1: Set Up the Environment
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Step 2: Define the Model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Step 4: Implement the Custom Training Loop with Accuracy Tracking
epochs = 5
for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        accuracy_metric.update_state(y_batch, logits)
    print(f'Epoch {epoch + 1}: Loss = {loss.numpy()} Accuracy = {accuracy_metric.result().numpy()}')
    accuracy_metric.reset_state()


### Exercise 3: Custom Callback for Advanced Logging

#### Objective: Implement a custom callback to log additional metrics and information during training.

#### Instructions:

1. Set up the environment and define the model, loss function, optimizer, and metric.

2. Create a custom callback to log additional metrics at the end of each epoch.

3. Implement the custom training loop with the custom callback.


In [25]:
# Write your code here
# Exercise 3: Custom Callback for Advanced Logging

class CustomLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\n✅ End of Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Accuracy = {logs['accuracy']:.4f}")

# Build and compile a simple model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train with callback
model.fit(x_train, y_train, epochs=3, batch_size=64, callbacks=[CustomLogger()])


Epoch 1/3
933/938 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8359 - loss: 0.5999
✅ End of Epoch 1: Loss = 0.3543, Accuracy = 0.9014
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8363 - loss: 0.5983
Epoch 2/3
920/938 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9480 - loss: 0.1849
✅ End of Epoch 2: Loss = 0.1734, Accuracy = 0.9503
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9480 - loss: 0.1847
Epoch 3/3
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9630 - loss: 0.1309
✅ End of Epoch 3: Loss = 0.1266, Accuracy = 0.9635
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9630 - loss: 0.1309


<details>
<summary>Click here for solution</summary> </br>

```python
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import Callback

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

# Step 2: Define the Model
model = Sequential([
    tf.keras.Input(shape=(28, 28)),  # Updated Input layer syntax
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10)
])

# Step 3: Define Loss Function, Optimizer, and Metric
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Step 4: Implement the Custom Callback
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f'End of epoch {epoch + 1}, loss: {logs.get("loss")}, accuracy: {logs.get("accuracy")}')

# Step 5: Implement the Custom Training Loop with Custom Callback
custom_callback = CustomCallback()

for epoch in range(5):
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        accuracy_metric.update_state(y_batch, logits)
    custom_callback.on_epoch_end(epoch, logs={'loss': loss.numpy(), 'accuracy': accuracy_metric.result().numpy()})
    accuracy_metric.reset_state()  # Updated method



### Exercise 4: Lab - Hyperparameter Tuning

#### Enhancement: Add functionality to save the results of each hyperparameter tuning iteration as JSON files in a specified directory.

#### Additional Instructions:

Modify the tuning loop to save each iteration's results as JSON files.

Specify the directory where these JSON files will be stored for easier retrieval and analysis of tuning results.


In [29]:
# Write your code here
# ✅ Exercise 4: Lab - Hyperparameter Tuning
import keras_tuner as kt
import json
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

def build_model(hp):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(
        units=hp.Int("units", min_value=32, max_value=512, step=32),
        activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

# Define tuner
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=1,
    directory="my_tuning_dir",
    project_name="mnist_tuning"
)

# Load and normalize MNIST data
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=3, validation_data=(x_val, y_val))

# Save best trial results to JSON
results = tuner.oracle.get_best_trials(num_trials=3)
for i, trial in enumerate(results):
    trial_result = {
        "trial_id": trial.trial_id,
        "score": trial.score,
        "params": trial.hyperparameters.values
    }
    with open(f"trial_{i+1}_results.json", "w") as f:
        json.dump(trial_result, f, indent=4)


Trial 3 Complete [00h 00m 55s]
val_accuracy: 0.9750000238418579

Best val_accuracy So Far: 0.9779999852180481
Total elapsed time: 00h 02m 44s


<details>
<summary>Click here for solution</summary> </br>

```python
!pip install keras-tuner
!pip install scikit-learn

import json
import os
import keras_tuner as kt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Step 1: Load your dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Step 2: Define the model-building function
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification example
    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 3: Initialize a Keras Tuner RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    executions_per_trial=1,  # Set how many executions per trial
    directory='tuner_results',  # Directory for saving logs
    project_name='hyperparam_tuning'
)

# Step 4: Run the tuner search (make sure the data is correct)
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=5)

# Step 5: Save the tuning results as JSON files
try:
    for i in range(10):
        # Fetch the best hyperparameters from the tuner
        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        
        # Results dictionary to save hyperparameters and score
        results = {
            "trial": i + 1,
            "hyperparameters": best_hps.values,  # Hyperparameters tuned in this trial
            "score": None  # Add any score or metrics if available
        }

        # Save the results as JSON
        with open(os.path.join('tuning_results', f"trial_{i + 1}.json"), "w") as f:
            json.dump(results, f)

except IndexError:
    print("Tuning process has not completed or no results available.")
 ```   

</details>


### Exercise 5: Explanation of Hyperparameter Tuning

**Addition to Explanation:** Add a note explaining the purpose of num_trials in the hyperparameter tuning context:


In [30]:
# Write your code here
# Exercise 5: Explanation of Hyperparameter Tuning

# The hyperparameter tuning is controlled by `max_trials`.
# Each trial represents a unique combination of hyperparameters.

# Example:
max_trials = 5

# This tells the tuner to try 5 different combinations of hyperparameters.
# The tuner will evaluate each and keep the best result.

# Other important parameters:
# - executions_per_trial: repeats training with the same params to reduce randomness.
# - objective: guides the tuner on what to maximize (e.g., val_accuracy).


<details>
<summary>Click here for solution</summary> </br>

```python
Explanation: "num_trials specifies the number of top hyperparameter sets to return. Setting num_trials=1 means that it will return only the best set of hyperparameters found during the tuning process."
 ```   

</details>


### Conclusion:

Congratulations on completing this lab! You have now successfully created, trained, and evaluated a simple neural network model using the Keras Functional API. This foundational knowledge will allow you to build more complex models and explore advanced functionalities in Keras.


Copyright © IBM Corporation. All rights reserved.
